In [2]:
import numpy as np
import pandas as pd
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


In [7]:
data = pd.read_csv('train.txt',sep=';')
data.columns = ['Text','Emotions']
data

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness
...,...,...
15994,i just had a very brief time in the beanbag an...,sadness
15995,i am now turning and i feel pathetic that i am...,sadness
15996,i feel strong and good overall,joy
15997,i feel like this was such a rude comment and i...,anger


In [32]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [33]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [34]:
#encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [35]:
# one hot encoding the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [36]:
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [37]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 18s 45ms/step - loss: 1.3584 - accuracy: 0.4819 - val_loss: 0.8500 - val_accuracy: 0.7266
Epoch 2/10
400/400 [==============================] - 18s 45ms/step - loss: 0.3661 - accuracy: 0.8839 - val_loss: 0.4951 - val_accuracy: 0.8334
Epoch 3/10
400/400 [==============================] - 18s 45ms/step - loss: 0.0613 - accuracy: 0.9845 - val_loss: 0.5048 - val_accuracy: 0.8425
Epoch 4/10
400/400 [==============================] - 18s 45ms/step - loss: 0.0249 - accuracy: 0.9952 - val_loss: 0.5264 - val_accuracy: 0.8338
Epoch 5/10
400/400 [==============================] - 18s 45ms/step - loss: 0.0164 - accuracy: 0.9972 - val_loss: 0.5633 - val_accuracy: 0.8313
Epoch 6/10
400/400 [==============================] - 18s 45ms/step - loss: 0.0142 - accuracy: 0.9970 - val_loss: 0.5880 - val_accuracy: 0.8331
Epoch 7/10
400/400 [==============================] - 18s 45ms/step - loss: 0.0127 - accuracy: 0.9974 - val_loss: 0.6512 - val_accuracy:

In [38]:
input_text = "She didnt't come today because she lost her dog yesterday!"



In [39]:
# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 44ms/step
['sadness']
